In [ ]:
###############
## Libraries ##
###############

import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras import datasets, layers, models, losses
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import keras

# Training

In [ ]:
######################################
## Import CIFAR10 data from Scratch ##
######################################
import pickle

# unpickle the binary files
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# paths to each batch of data
batch1 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_1")
batch2 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_2")
batch3 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_3")
batch4 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_4")
batch5 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_5")
meta = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/batches.meta")
test = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/test_batch")

# separate labels and image data from each batch
y_train1 = batch1[b'labels']
x_train1 = batch1[b'data']
y_train2 = batch2[b'labels']
x_train2 = batch2[b'data']
y_train3 = batch3[b'labels']
x_train3 = batch3[b'data']
y_train4 = batch4[b'labels']
x_train4 = batch4[b'data']
y_train5 = batch5[b'labels']
x_train5 = batch5[b'data']

# concatenate into big training and testing arrays
y_train = np.concatenate((y_train1, y_train2, y_train3, y_train4, y_train5))
x_train = np.concatenate((x_train1, x_train2, x_train3, x_train4, x_train5), axis=0)

def shuffle_in_unison(x, y):
    assert x.shape[0] == y.shape[0]
    shuffled_x = np.empty(x.shape, dtype=x.dtype)
    shuffled_y = np.empty(y.shape, dtype=y.dtype)
    
    # If rerunning: 
    # permutation = np.loadtxt('/scratch/gpfs/eysu/src_data/cifar-10-batches-py/permutation.csv', delimiter=',').astype(np.int64)
    
    # If repermuting: 
    # permutation = np.random.permutation(y.shape[0])
    
    for old_index, new_index in enumerate(permutation):
        shuffled_x[new_index] = x[old_index]
        shuffled_y[new_index] = y[old_index]

    # IF I WANT TO RUN THIS AGAIN DONT REPERMUTE OR WILL LOSE THIS ORDERING
#     np.savetxt('/scratch/gpfs/eysu/src_data/cifar-10-batches-py/permutation.csv', permutation, delimiter=',')
    return shuffled_x, shuffled_y

x_train, y_train = shuffle_in_unison(x_train, y_train)

y_test = test[b'labels']
x_test = test[b'data']

In [ ]:
#################################################
## Preprocess data by reshaping and separating ##
#################################################
labels = ['airplane',  # index 0
          'automobile',  # index 1
          'bird',  # index 2 
          'cat',  # index 3 
          'deer',  # index 4
          'dog',  # index 5
          'frog',  # index 6 
          'horse',  # index 7 
          'ship',  # index 8 
          'truck']  # index 9

# Further break training data into train / validation sets 
# put 5000 into validation set and keep remaining 45,000 for train
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

# reshape data to match dimensions of cifar10.load_data
x_train = x_train.reshape(45000, 3, 32, 32)
x_train = x_train.transpose(0, 2, 3, 1)
x_train = x_train.astype('float32')
x_train /= 255

# y_train = tf.keras.utils.to_categorical(y_train, 10)

x_valid = x_valid.reshape(5000, 3, 32, 32)
x_valid = x_valid.transpose(0, 2, 3, 1)
x_valid = x_valid.astype('float32')
x_valid /= 255
y_valid = tf.keras.utils.to_categorical(y_valid, 10)

x_test = x_test.reshape(10000, 3, 32, 32)
x_test = x_test.transpose(0, 2, 3, 1)
x_test = x_test.astype('float32')
x_test /= 255
y_test = tf.keras.utils.to_categorical(y_test, 10)

# assert dimensions of data
print("TRAINING DATA")
print(x_train.shape)
print(y_train.shape)

print("VALIDATION DATA")
print(x_valid.shape)
print(y_valid.shape)

print("TESTING DATA")
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Examine any image

# Image index, you can pick any number between 0 and 44,999
img_index = 10
label_index = y_train[img_index]
# Print the label, for example 2 Pullover
print("y = " + str(label_index) + " (" +(labels[label_index]) + ")")
plt.imshow(x_train[img_index])
plt.show()

In [ ]:
##############################################################
## This cell runs the iterated learning training procedure. ##
##############################################################

# Number of iterations in the serial reproduction
MAX_ITER = 25
# Number of epochs per training run
EPOCHS = 10

for iteration in range(0,MAX_ITER):
    # If iteration is seed, train on original target vectors, else, train on y_hat from time t-1
    if iteration == 0:
        # One-hot encode the labels
        y_train = tf.keras.utils.to_categorical(y_train, 10)
        mpth = 'model.weights.best.hdf5'
        y_hat_test_name = 'y_hat_test_seed'
        y_hat_train_name = 'y_hat_train_seed'      
    elif iteration > 0:
        # Key step: set new targets as y_hat
        y_train = y_hat    
        mpth = 'model.weights.best.' + 'iter' + str(iteration) + '.hdf5'
        y_hat_test_name = 'y_hat_test_' + 'iter' + str(iteration)
        y_hat_train_name = 'y_hat_train_' + 'iter' + str(iteration)

    # Define the model: a small CNN model (could probably be done outside loop)
    model = tf.keras.Sequential()

    # Must define the input shape in the first layer of the neural network
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # Take a look at the model summary
    # model.summary()

    # define optimization and energy parameters
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Save checkpoints
    checkpointer = ModelCheckpoint(filepath= '/scratch/gpfs/eysu/CIFAR10_results/og_training/' + mpth, verbose = 1, save_best_only=True) #True
    # Train the model
    model.fit(x_train,
             y_train,
             batch_size=64,
             epochs=EPOCHS,
             validation_data=(x_valid, y_valid),
             callbacks=[checkpointer])

    # Load the weights with the best validation accuracy
    y_hat = model.predict(x_train) #feed back serial reproduction targets
    y_hat_test = model.predict(x_test)
    
    model.load_weights('/scratch/gpfs/eysu/CIFAR10_results/og_training/' + mpth)
    # Evaluate the model on test set
    score = model.evaluate(x_test, y_test, verbose=0)
    # Print test accuracy
    print('\n', 'Test accuracy:', score[1])

    # Save results for each iteration in the serial reproduction chain
    np.save('/scratch/gpfs/eysu/CIFAR10_results/og_training/' + y_hat_train_name + '.npy', y_train)
    print('/scratch/gpfs/eysu/CIFAR10_results/og_training/' + y_hat_train_name)
#     np.save('weights/' + y_hat_test_name + DATASET + REGIME + '.npy', y_test)
    np.save('/scratch/gpfs/eysu/CIFAR10_results/og_training/' + y_hat_test_name + '.npy', y_hat_test)
    print('/scratch/gpfs/eysu/CIFAR10_results/og_training/' + y_hat_test_name)



# Visualize

In [ ]:
############################################################################################
## Read prediction softmax layer activations for all train set images, and all iterations ##
############################################################################################

MAX_ITER = 25

# Build arrays of dimensions: N training images X L labels X P iterations
y_hat_train_arr = np.zeros([y_train.shape[0], len(labels), MAX_ITER])

for i in range(MAX_ITER):
    if i == 0:
        y_hat_train_name = 'y_hat_train_seed'
    else:
        y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)
        
    # Load test set softmax outputs 
    yhtr = np.load('/scratch/gpfs/eysu/CIFAR10_results/og_training/' + y_hat_train_name + '.npy')

    # The first time through, use binary weight vectors to save correct class array
    if i == 0:
        true_class_tr = np.nonzero(yhtr)[1]
        
    y_hat_train_arr[:, :, i] = yhtr

print(y_hat_train_arr.shape)
# (45000, 10, 12)

In [ ]:
########################
## Visualize N images ##
########################

from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.filterwarnings('ignore')
import matplotlib.backends.backend_pdf

def visualize_N_images(CLASS, NUM_IMAGES, rank):
    pdf = matplotlib.backends.backend_pdf.PdfPages("Outputs/least_confusing_" + str(labels[CLASS]) + "s.pdf")
    # NUM_IMAGES softmax outputs least and most confusing images for CLASS
    class_data_least_confusing = class_data[rank[:NUM_IMAGES]]
    class_data_most_confusing = class_data[rank[-NUM_IMAGES:]]


    # extract NUM_IMAGES images of CLASS data from the overall training images
    class_images = x_train[np.where(true_class_tr == CLASS)]

    images_least_confusing = class_images[rank[:NUM_IMAGES]]
    images_most_confusing = class_images[rank[-NUM_IMAGES:]]

    # visualize each image
    print(str(NUM_IMAGES) + " least confusing " + str(labels[CLASS]) + "s")
    for i in range(NUM_IMAGES):
        current = class_data_least_confusing[i]

        figure = plt.figure(figsize=(40, 40))
        # plot image
        ax1 = figure.add_subplot(8, 8, 1, xticks=[], yticks=[])
        im1 = ax1.imshow(images_least_confusing[i])
        ax1.set_title("Image: " + str(labels[CLASS]))

        # plot weights graph
        ax2 = figure.add_subplot(8, 8, 2)
        im2 = ax2.imshow(current.T, cmap='Wistia')

        divider = make_axes_locatable(ax2)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        cbar = figure.colorbar(im2, cax=cax, orientation='vertical', ticks=[0, 1])
        cbar.ax.set_yticklabels(['0', '1'])

        ax2.set(xlabel='Classes', ylabel='Iterations', title='Softmax Outputs')
        ax2.set_xticks(ticks=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
        ax2.set_xticklabels(labels, rotation='vertical')

        # plot correlation graph
        corr_arr = np.corrcoef(current.T)

        ax3 = figure.add_subplot(8, 8, 3)
        im3 = ax3.imshow(corr_arr, cmap='Wistia')
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        cbar = figure.colorbar(im2, cax=cax, orientation='vertical', ticks=[0, 1])
        cbar.ax.set_yticklabels(['0', '1'])
        ax3.set_title("Correlation Matrix")

        # plot eigenvalues graph
        eigs, _ = np.linalg.eig(corr_arr)
        num_nonzero = np.count_nonzero(np.around(eigs, 2))

        if (num_nonzero == 1):
            title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzero)"
        else:
            title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzeros)"

        ax4 = figure.add_subplot(8, 8, 4)
        im4 = ax4.plot(eigs, marker='o')
        ax4.set(xlabel="PC Number", xlim=[0,len(eigs)], ylim=[0,MAX_ITER], title=title_str)
        
        pdf.savefig(figure, bbox_inches='tight')
        plt.show()
        
    pdf.close()

    pdf = matplotlib.backends.backend_pdf.PdfPages("Outputs/most_confusing_" + str(labels[CLASS]) + "s.pdf")
    print(str(NUM_IMAGES) + " most confusing " + str(labels[CLASS]) + "s")
    for i in range(NUM_IMAGES):
        current = class_data_most_confusing[i]

        figure = plt.figure(figsize=(40, 40))
        # plot image
        ax1 = figure.add_subplot(8, 8, 1, xticks=[], yticks=[])
        im1 = ax1.imshow(images_most_confusing[i])
        ax1.set_title("Image: " + str(labels[CLASS]))

        # plot weights graph
        ax2 = figure.add_subplot(8, 8, 2)
        im2 = ax2.imshow(current.T, cmap='Wistia')

        divider = make_axes_locatable(ax2)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        figure.colorbar(im2, cax=cax, orientation='vertical', ticks=[0, 1])

        ax2.set(xlabel='Classes', ylabel='Iterations', title='Softmax Outputs')
        ax2.set_xticks(ticks=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
        ax2.set_xticklabels(labels, rotation='vertical')

        # plot correlation graph
        corr_arr = np.corrcoef(current.T)

        ax3 = figure.add_subplot(8, 8, 3)
        im3 = ax3.imshow(corr_arr, cmap='Wistia')
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        cbar = figure.colorbar(im2, cax=cax, orientation='vertical', ticks=[0, 1])
        cbar.ax.set_yticklabels(['0', '1'])
        ax3.set_title("Correlation Matrix")

        # plot eigenvalues graph
        eigs, _ = np.linalg.eig(corr_arr)
        num_nonzero = np.count_nonzero(np.around(eigs, 2))

        if (num_nonzero == 1):
            title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzero)"
        else:
            title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzeros)"

        ax4 = figure.add_subplot(8, 8, 4)
        im4 = ax4.plot(eigs, marker='o')
        ax4.set(xlabel="PC Number", xlim=[0,len(eigs)], ylim=[0, MAX_ITER], title=title_str)

        
        pdf.savefig(figure, bbox_inches='tight')
        plt.show()
    pdf.close()

In [ ]:
##############################################
## Rank by difference to binary seed vector ##
##############################################
print("Rank method: Binary seed vector difference")

for i in range(10):
    # choose which class to analyze
    CLASS = i

    # take difference of predictionn probability along each row for indicated class
    class_data = y_hat_train_arr[np.where(true_class_tr == CLASS), :, :].squeeze()

    # calculate differences in weights for each image between true and final iter
    diff_arr = class_data[:, CLASS, 0] - class_data[:, CLASS, MAX_ITER - 1]

    #rank images by magnitude of difference from correct prediction, default ascending
    seed_diff_rank = np.argsort(diff_arr)
    
    visualize_N_images(i, 100, seed_diff_rank)

# Scratch work